# Model Optimization

In this notebook, we will further optimize the performance of our selected models (Logistic Regression and SVM) using advanced techniques such as feature engineering, handling imbalanced data, ensemble methods, and further hyperparameter tuning.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

In [2]:
# Load the preprocessed dataset
data_path = "../data/processed/cleaned_data.csv"
df = pd.read_csv(data_path)

In [3]:
# Separate features and target variable
X = df.drop(columns=['customerID', 'Churn'])
y = df['Churn']

In [4]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set size:", X_train.shape)
print("Testing set size:", X_test.shape)

Training set size: (5634, 19)
Testing set size: (1409, 19)


## Feature Engineering

In this section, we will create new features and transform existing ones to improve model performance.


In [5]:
# Example: Creating interaction terms
X_train['MonthlyTenureInteraction'] = X_train['MonthlyCharges'] * X_train['tenure']
X_test['MonthlyTenureInteraction'] = X_test['MonthlyCharges'] * X_test['tenure']

# Example: Transforming categorical variables using OneHotEncoder
categorical_features = ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService',
                        'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
                        'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']

numeric_features = ['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges', 'MonthlyTenureInteraction']

In [6]:
# Preprocessing pipelines for numerical and categorical features
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [7]:
# Apply transformations
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

## Handling Imbalanced Data

We will use SMOTE (Synthetic Minority Over-sampling Technique) to address class imbalance.


In [8]:
# Apply SMOTE to training data
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_transformed, y_train)

# Check the distribution of the target variable after resampling
print("Distribution of target variable after SMOTE:")
print(y_train_resampled.value_counts())

C:\Users\Kahla\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\Kahla\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


Distribution of target variable after SMOTE:
Churn
0    4138
1    4138
Name: count, dtype: int64


## Ensemble Methods

We will explore ensemble methods such as Random Forest and Gradient Boosting to improve model performance.


In [9]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Initialize ensemble models
ensemble_models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42)
}

In [11]:
# Function to evaluate model performance
def evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    return accuracy, precision, recall, f1, y_pred

In [12]:
# Dictionary to store evaluation results
ensemble_results = {}

In [13]:
# Evaluate each ensemble model
for model_name, model in ensemble_models.items():
    accuracy, precision, recall, f1, y_pred = evaluate_model(model, X_train_resampled, y_train_resampled, X_test_transformed, y_test)
    ensemble_results[model_name] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': confusion_matrix(y_test, y_pred)
    }